<a href="https://colab.research.google.com/github/Soniartha/Trading-Strategy/blob/main/CNN_Jason_Brownlee_Univariate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Multivariate CNNs Models
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [6]:
#Define sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range (len(in_seq1))])

In [7]:
#convert into rows and columns 
in_seq1 = in_seq1.reshape((len(in_seq1),1))
in_seq2 = in_seq2.reshape((len(in_seq2),1))
out_seq = out_seq.reshape((len(out_seq),1))

In [14]:
#Horizontal stack
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [20]:
def split_sequence(sequence, n_steps):
  X, y = list (), list ()
  for i in range (len(sequence)):
    end_ix = i + n_steps
    if end_ix > len(sequence): #khusus untuk multi variate tidak -1
      break
    seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix-1, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [10]:
#Define n_steps
n_steps = 3

In [21]:
#Split into input and output in this pattern (X,y)
X,y = split_sequence(dataset, n_steps)

In [22]:
#Display the data 
for i in range(len(X)):
  print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [26]:
#n_feature for 3D Structure
n_features = X.shape[2]
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [25]:
#Define model
model = Sequential()
model.add(Conv1D(filters= 64, kernel_size=2, activation= 'relu', input_shape= (n_steps, n_features)))
model.add(MaxPooling1D(pool_size= 2))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer = 'adam', loss= 'mse')

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


In [31]:
#Fit model
model.fit(X, y, epochs= 1000, verbose= 0)

In [33]:
#Demonstrate prediction
x_input = array([[80,85],[90,95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
y_hat = model.predict(x_input, verbose=0)
print(y_hat)

[[206.59293]]
